# EXTRACTION
## Data Source
- Use Events API: https://api.predicthq.com/v1/events?
## Methodology
### Data Analysis - View Data
1. Convert to json format and read json. Determine which variables are useful and required for the deliverable.

FINDINGS: The API allows user to call for 50 entries/page, and up to 100 pages. i.e. Each time a call is made, there are only 50 entry results. 
### Data Analysis - Deetermine and Extract Data Required

- Number of entries required: 5000. 
    1. Loop and set offset at every 50 interval (0, 50, 100, 150 etc until it reaches 4950. 5000 results max from API.)
- Variables required: id, country, category, title, start_date, end_date, country, location
    2. Use pandas to filter for the variables required. 
        - *location is the coordinates

# TRANSFORMATION
- Concatenate the dataframes together (from all 100 pages) and save the final dataframe to a variable.

# LOAD
- Load data into PostSQL

-------------------------------

# EXTRACTION

In [1]:
# Dependencies
import requests
import pprint
import json
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize 
from sqlalchemy import create_engine

# Credential File: py_config.py containing variable ACCESS_TOKEN = "xxxxxxxxx"
import py_config

## Data Analysis - View Data
Convert to json format and read json. Determine which variables are useful and required for the deliverable.

In [2]:
# Connect to API url and get data
# Variable ACCESS_TOKEN is referenced in py_config.py file and is in list .gitignore
response = requests.get(
    url ="https://api.predicthq.com/v1/events?",
    headers={
      "Authorization": f"Bearer {py_config.ACCESS_TOKEN}",
      "Accept": "application/json"
    },
    params={
      "limit":50,
      "country": "AU",
      "start": "2021-01-01",
      "end": "2022-12-31"
    }
)

In [3]:
# Convert data to json format (all data)
# Save to variable 
data = response.json()

# Print json (formatted) and analyse which variables to use for deliverable
print(json.dumps(data, indent=4, sort_keys=True))

{
    "count": 5000,
    "next": "https://api.predicthq.com/v1/events/?country=AU&end=2022-12-31&limit=50&offset=50&start=2021-01-01",
    "overflow": true,
    "previous": null,
    "results": [
        {
            "aviation_rank": 0,
            "brand_safe": true,
            "category": "conferences",
            "country": "AU",
            "description": "",
            "duration": 32400,
            "end": "2021-07-29T08:00:00Z",
            "entities": [
                {
                    "entity_id": "J2CejquZgY9Nrrr4bWwc54",
                    "formatted_address": "192 Wellington Parade\nMelbourne VIC 3002\nAustralia",
                    "name": "Pullman Melbourne On The Park",
                    "type": "venue"
                }
            ],
            "first_seen": "2021-01-25T14:26:48Z",
            "id": "3EmLG6VBfRz6MWr2CP",
            "labels": [
                "conference",
                "education",
                "technology"
            ],
          

In [4]:
# Analyse keys. Show only 1 result at this stage for analysis.
data['results'][0]

{'relevance': 1.0,
 'id': '3EmLG6VBfRz6MWr2CP',
 'title': 'Victoria IT Leadership Summit',
 'description': '',
 'category': 'conferences',
 'labels': ['conference', 'education', 'technology'],
 'rank': 42,
 'local_rank': 55,
 'aviation_rank': 0,
 'phq_attendance': 400,
 'entities': [{'formatted_address': '192 Wellington Parade\nMelbourne VIC 3002\nAustralia',
   'entity_id': 'J2CejquZgY9Nrrr4bWwc54',
   'name': 'Pullman Melbourne On The Park',
   'type': 'venue'}],
 'duration': 32400,
 'start': '2021-07-28T23:00:00Z',
 'end': '2021-07-29T08:00:00Z',
 'updated': '2021-02-04T11:58:27Z',
 'first_seen': '2021-01-25T14:26:48Z',
 'timezone': 'Australia/Melbourne',
 'location': [144.983394, -37.815795],
 'scope': 'locality',
 'country': 'AU',
 'place_hierarchies': [['6295630',
   '6255151',
   '2077456',
   '2145234',
   '7839805',
   '6952202'],
  ['6295630', '6255151', '2077456', '2145234', '7839805', '2158177']],
 'state': 'active',
 'brand_safe': True,
 'private': False}

## Data Analysis - Determine and Extract Data Required
- Number of entries required: 5000. 
    1. Loop and set offset at every 50 interval (0, 50, 100, 150 etc until it reaches 4950. 5000 results max from API.)
- Variables required: id, country, category, title, start_date, end_date, country, location
    2. Use pandas to filter for the variables required. 
        - *location is the coordinates

In [5]:
# Test for loop for accessing numerous entries.
for i in range (0,500,50):
    print(i)

0
50
100
150
200
250
300
350
400
450


In [6]:
# Create variable "entries" to store the list of dictionaries; 
# Each loop will contain a dictionary (as each pandas dataframe is a dictionaries). 
events_entries=[]

# Do a 'for loop' which loops from index 0 to 5000, at muliples of 50 (as API page can only call 50 entries max each time)
for i in range(0,1500,50):

    response = requests.get(
        url=f"https://api.predicthq.com/v1/events?offset={i}&limit=50",
        headers={
        "Authorization": f"Bearer {py_config.ACCESS_TOKEN}",
        "Accept": "application/json"
        },
        params={
            "country": "AU",
            "start": "2021-01-01",
            "end": "2022-12-31"
        }
    )

    # Save response to variable "data" 
    data = response.json()
  
    # Save to variable "events_df" 
    events_df = pd.json_normalize(data, ['results'], errors='ignore')
    # print(events_df.head(3))

    # Create function to store variable 'name' 
    def getEntitiesName(entities):
        try:
            return entities[0]['name']
        except:
            return 'no name'
    
    events_df['name'] = events_df.entities.apply(getEntitiesName)


    # Create function to store variable 'formatted_address' 
    def getEntitiesAddress(entities):
        try:
            return entities[0]['formatted_address']
        except:
            return 'no address'
    events_df['formatted_address'] = events_df.entities.apply(getEntitiesAddress)

    # Create function to store variable 'venue_name' 
    def getEntitiesVenue(entities):
        try:
            return entities[1]['name']
        except IndexError:
            return 'no venue'
    events_df['venue_name'] = events_df.entities.apply(getEntitiesVenue)

    # Extract out only required variables (column headings)
    events_df = events_df[["id","title","description","category","start","end","country","location","rank","name","venue_name",                 "formatted_address"]]
     
     # append to list 'events_entries
    events_entries.append(events_df)

    # TEST
    # print(events_entries)

# TRANSFORMATION
- Concatenate the dataframes together (from all 100 pages) and save the final dataframe to a variable.

In [7]:
# Concatenate all the dictionaries within list "events_entries". 
# i.e. Convert list to contain the data in the 1 dictionary.
# Save into variable "events_entries_df"
events_entries_df = pd.concat(events_entries)

# Rename columns
events_entries_df = events_entries_df.rename(columns={'start': 'start_date','end':'end_date','location':'coords','name':'title2'})
# events_entries_df

# Drop column title2
events_entries_df = events_entries_df.drop(columns={'title2'})
events_entries_df

,id,title,description,category,start_date,end_date,country,coords,rank,venue_name,formatted_address
0,3EmLG6VBfRz6MWr2CP,Victoria IT Leadership Summit,,conferences,2021-07-28T23:00:00Z,2021-07-29T08:00:00Z,AU,"[144.983394, -37.815795]",42,no venue,192 Wellington Parade\nMelbourne VIC 3002\nAus...
1,6Cz4T2oXQHXaYmQPns,Democracy. Are You In?,Democracy. Are You In? is a contemporary exhib...,expos,2021-07-28T23:00:00Z,2021-07-29T07:00:00Z,AU,"[149.129959, -35.30196]",45,Museum of Australian Democracy,no address
2,8itJVSBjPKeG6RFKSg,The Polished Opal,The workshop starts with a short talk about op...,community,2021-07-28T23:00:00Z,2021-07-29T02:00:00Z,AU,"[150.333328, -33.702448]",0,no venue,no address
3,APnTSQufUMLXs7LKVp,Indigenous Stock Workers and Rodeo Riders Disp...,An informative and visual display in recogniti...,expos,2021-07-28T23:00:00Z,2021-07-29T06:00:00Z,AU,"[141.081268, -17.668348]",45,no venue,no address
4,BqmbGD2ySaYL3tuUUx,"Truth, Power and a Free Press","Truth, Power and a Free press is a compelling ...",expos,2021-07-28T23:00:00Z,2021-07-29T07:00:00Z,AU,"[149.129959, -35.30196]",45,Museum of Australian Democracy,no address
...,...,...,...,...,...,...,...,...,...,...,...
45,2AQXBXa4m8YhuQcF6y,Lismore Organic Market,The Lismore Organic Market is a weekly market ...,community,2021-07-19T21:30:00Z,2021-07-20T01:30:00Z,AU,"[153.272963, -28.797029]",45,no venue,no address
46,PUpHhxFcr7gUH2BEoU,Bowen parkrun,"Parkrun is a free, weekly, timed five kilometr...",sports,2021-07-19T21:00:00Z,2021-07-19T22:30:00Z,AU,"[148.252191, -19.986951]",45,no venue,no address
47,R7S3Fi2hCzLtCdSso3,Making Meditation Mainstream Free Beach Medita...,Making Meditation Mainstream is a community mo...,community,2021-07-19T20:30:00Z,2021-07-19T21:00:00Z,AU,"[153.120592, -26.680521]",45,no venue,no address
48,GEeFvEmrv9857jA6gk,Screen Coach | Acting For Screen Class,Want to learn a new creative skill or develop ...,community,2021-07-19T17:30:00Z,2021-07-20T08:30:00Z,AU,"[151.27134, -33.900037]",0,no venue,no address


In [8]:
events_entries_df.columns

Index(['id', 'title', 'description', 'category', 'start_date', 'end_date',
       'country', 'coords', 'rank', 'venue_name', 'formatted_address'],
      dtype='object')

In [9]:
# Check dtypes
events_entries_df.dtypes

id                   object
title                object
description          object
category             object
start_date           object
end_date             object
country              object
coords               object
rank                  int64
venue_name           object
formatted_address    object
dtype: object

In [10]:
# Analyse the category types and entries within each category.
events_entries_df['category'].value_counts()

community          493
expos              442
sports             275
performing-arts    181
conferences         41
festivals           41
concerts            26
observances          1
Name: category, dtype: int64

# LOAD

In [11]:
# Connect to local database
# for everyone else
rds_connection_string = "postgres:postgres@localhost:5432/events_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
# Diane's use only
# rds_connection_string = "postgres:309Malanday!@localhost:5432/events_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
#### IN POSGRES DATABASE, RUN QUERY: 
# Create a database called 'events_db' in postgres database. Add the tables (table headings defined) into postgres database as per schema.sql

# Run code to ensure connection is established and table has been created on postgres.
engine.table_names()

['events_table']

In [ ]:
# Load data events_entries_df into postgres database 'events_db', in the table 'events_table'
events_entries_df.to_sql(name='events_table', con=engine, if_exists='append', index=False)


In [14]:
# Run code to check connection is established and data is reading out from postgres database
events_info = pd.read_sql_table('events_table', engine) 
events_info

,id,title,description,category,start_date,end_date,country,coords,rank,venue_name,formatted_address
0,UZGsZMeY5YegR7MMdD,Messy Play Sessions,The Busy Peacock is a place to come and let th...,community,2021-07-20 23:30:00+00:00,2021-07-21 00:15:00+00:00,AU,"{145.18578,-38.261048}",0,no venue,no address
1,eQN8f47RzTXvibpxY7,INVOLVING FAMILIES AND OTHERS IN THE CARE OF Y...,This workshop will explore the benefits and ba...,conferences,2021-07-20 23:30:00+00:00,2021-07-21 06:30:00+00:00,AU,"{144.947904,-37.780223}",0,no venue,no address
2,57NrcFdTDNZcHGZ5Jw,Future of Financial Services,,conferences,2021-07-20 23:00:00+00:00,2021-07-21 08:00:00+00:00,AU,"{144.959213,-37.82341}",46,The Pub at Crown,8 Whiteman St Southbank VIC 3006
3,5gt6YrELDVgyH9H6N4,The Trevor Kennedy Collection: Highlights,Discover objects of rare beauty and items of c...,expos,2021-07-20 23:00:00+00:00,2021-07-21 07:00:00+00:00,AU,"{149.119532,-35.292481}",45,National Museum of Australia,no address
4,65grzvvvAQcmZH7rim,Gunnedah Saleyards,Experience one of the largest stock selling ce...,expos,2021-07-20 23:00:00+00:00,2021-07-21 01:00:00+00:00,AU,"{150.224511,-30.958274}",45,no venue,no address
...,...,...,...,...,...,...,...,...,...,...,...
1495,h8Tgn52FZ9ehPwsngi,Queensland NPL 2 Youth - Sunshine Coast U23 vs...,,sports,2021-07-11 04:00:00+00:00,2021-07-11 04:00:00+00:00,AU,"{153.119022,-26.731251}",40,no venue,Milieu Place\nKawana Waters\nAustralia
1496,kWzqFJbmKmN2tU82rQ,Private photography class in Sydney,"Develop your composition and shooting skills, ...",community,2021-07-11 04:00:00+00:00,2021-07-11 07:00:00+00:00,AU,"{151.208565,-33.858768}",0,no venue,no address
1497,7URTbV4WsM39x4UTsC,Queensland NPL Youth League - Brisbane Roar U2...,,sports,2021-07-11 03:45:00+00:00,2021-07-11 03:45:00+00:00,AU,"{153.262453,-27.532772}",40,no venue,"60 - 76 Waterloo Street, Cleveland\nBrisbane\n..."
1498,9CrCqk5dQyqxEC7s3k,Queensland NPL 2 Youth - Brisbane City U23 vs ...,,sports,2021-07-11 03:45:00+00:00,2021-07-11 03:45:00+00:00,AU,"{153.069671,-27.590388}",46,no venue,23 Dew St.\nRuncorn QLD 4113\nAustralia


# ETL PROCESS COMPLETED

## ADDITIONAL TRANSFORMATION
In hindsight, the following code (cleaning data) should have been done within transformation but all team members had already loaded code to Postgres database at that stage of the project. 

Thus, the decision was to do some minor cleaning (as seen below) in app.py (Flask), after the data was pulled back out from Postgres.

In [15]:
events_info['coords']

0        {145.18578,-38.261048}
1       {144.947904,-37.780223}
2        {144.959213,-37.82341}
3       {149.119532,-35.292481}
4       {150.224511,-30.958274}
                 ...           
1495    {153.119022,-26.731251}
1496    {151.208565,-33.858768}
1497    {153.262453,-27.532772}
1498    {153.069671,-27.590388}
1499    {153.188151,-27.665874}
Name: coords, Length: 1500, dtype: object

In [16]:
# Removing '{' and '}' from 'coords' column
events_info['coords'] = events_info.coords.str.lstrip('{')
events_info['coords'] = events_info.coords.str.rstrip('}')

In [17]:
events_info['coords']

0        145.18578,-38.261048
1       144.947904,-37.780223
2        144.959213,-37.82341
3       149.119532,-35.292481
4       150.224511,-30.958274
                ...          
1495    153.119022,-26.731251
1496    151.208565,-33.858768
1497    153.262453,-27.532772
1498    153.069671,-27.590388
1499    153.188151,-27.665874
Name: coords, Length: 1500, dtype: object

In [18]:
# Final check
events_info

,id,title,description,category,start_date,end_date,country,coords,rank,venue_name,formatted_address
0,UZGsZMeY5YegR7MMdD,Messy Play Sessions,The Busy Peacock is a place to come and let th...,community,2021-07-20 23:30:00+00:00,2021-07-21 00:15:00+00:00,AU,"145.18578,-38.261048",0,no venue,no address
1,eQN8f47RzTXvibpxY7,INVOLVING FAMILIES AND OTHERS IN THE CARE OF Y...,This workshop will explore the benefits and ba...,conferences,2021-07-20 23:30:00+00:00,2021-07-21 06:30:00+00:00,AU,"144.947904,-37.780223",0,no venue,no address
2,57NrcFdTDNZcHGZ5Jw,Future of Financial Services,,conferences,2021-07-20 23:00:00+00:00,2021-07-21 08:00:00+00:00,AU,"144.959213,-37.82341",46,The Pub at Crown,8 Whiteman St Southbank VIC 3006
3,5gt6YrELDVgyH9H6N4,The Trevor Kennedy Collection: Highlights,Discover objects of rare beauty and items of c...,expos,2021-07-20 23:00:00+00:00,2021-07-21 07:00:00+00:00,AU,"149.119532,-35.292481",45,National Museum of Australia,no address
4,65grzvvvAQcmZH7rim,Gunnedah Saleyards,Experience one of the largest stock selling ce...,expos,2021-07-20 23:00:00+00:00,2021-07-21 01:00:00+00:00,AU,"150.224511,-30.958274",45,no venue,no address
...,...,...,...,...,...,...,...,...,...,...,...
1495,h8Tgn52FZ9ehPwsngi,Queensland NPL 2 Youth - Sunshine Coast U23 vs...,,sports,2021-07-11 04:00:00+00:00,2021-07-11 04:00:00+00:00,AU,"153.119022,-26.731251",40,no venue,Milieu Place\nKawana Waters\nAustralia
1496,kWzqFJbmKmN2tU82rQ,Private photography class in Sydney,"Develop your composition and shooting skills, ...",community,2021-07-11 04:00:00+00:00,2021-07-11 07:00:00+00:00,AU,"151.208565,-33.858768",0,no venue,no address
1497,7URTbV4WsM39x4UTsC,Queensland NPL Youth League - Brisbane Roar U2...,,sports,2021-07-11 03:45:00+00:00,2021-07-11 03:45:00+00:00,AU,"153.262453,-27.532772",40,no venue,"60 - 76 Waterloo Street, Cleveland\nBrisbane\n..."
1498,9CrCqk5dQyqxEC7s3k,Queensland NPL 2 Youth - Brisbane City U23 vs ...,,sports,2021-07-11 03:45:00+00:00,2021-07-11 03:45:00+00:00,AU,"153.069671,-27.590388",46,no venue,23 Dew St.\nRuncorn QLD 4113\nAustralia
